# What's in an Avocado Toast: A Supply Chain Analysis

![](avocado_wallpaper.jpeg)

You find yourself in London, crafting a delectable avocado toast, a dish that has risen dramatically in popularity on breakfast menus since the 2010s. This straightforward recipe requires just five ingredients: a ripe avocado, half a lemon, a generous pinch of salt flakes, two slices of sourdough bread, and a good drizzle of extra virgin olive oil. Most of these ingredients are now staples in grocery stores, and as you will find with this project, that is no small feat!

In this project, you'll conduct a supply chain analysis of three ingredients used in avocado toast using the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin.
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple category tags in the `categories_tags` column.
There is a column in each CSV file called `origins_tags`, which contains strings for the country of origin of each item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

[Open Food Facts database](https://world.openfoodfacts.org/)

In [1]:
import pandas as pd

In [50]:
# read in datasets needed

file_path = "/Users/Uche1/Desktop/Data/GitHub/supply_chain_analysis/data"

file_name = ["avocado.csv", "olive_oil.csv", "sourdough.csv", "relevant_olive_oil_categories.txt", "relevant_sourdough_categories.txt",
             "relevant_avocado_categories.txt"]

columns_to_read = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 
                   'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags']

avocado = pd.read_csv(f"{file_path}/{file_name[0]}", delimiter='\t', usecols= columns_to_read)

olive_oil = pd.read_csv(f"{file_path}/{file_name[1]}", delimiter='\t', usecols= columns_to_read)

sourdough = pd.read_csv(f"{file_path}/{file_name[2]}", delimiter='\t', usecols= columns_to_read)

sourdough_categories = pd.read_csv(f"{file_path}/{file_name[4]}", delimiter='\t', header= None)

olive_oil_categories = pd.read_csv(f"{file_path}/{file_name[3]}", delimiter='\t', header= None)

avocado_categories = pd.read_csv(f"{file_path}/{file_name[5]}", delimiter='\t', header= None)

/var/folders/fg/7ll5hhdd1817fzv12j0r4bfh0000gp/T/ipykernel_68521/1615206906.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  olive_oil = pd.read_csv(f"{file_path}/{file_name[1]}", delimiter='\t', usecols= columns_to_read)


In [51]:
avocado_categories = avocado_categories.rename(columns = {0: "categories_tags"}).reset_index(drop=True)
sourdough_categories = sourdough_categories.rename(columns={0:"categories_tags"}).reset_index(drop=True)
olive_oil_categories = olive_oil_categories.rename(columns = {0: "categories_tags"}).reset_index(drop=True)

In [55]:
type(avocado_categories)

pandas.core.frame.DataFrame

In [34]:
#drop all na rows 
avocado2 = avocado.loc[avocado["origins_tags"].notna()]

# turn columns having multiple categories and origin into a list of categories and origin
avocado3 = avocado2.copy()
avocado3["categories_tags"] = avocado2["categories_tags"].str.split(",")
avocado3["origins"] = avocado2["origins"].str.split(",")

avocado3

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
14,4016249238155,de,NaN,135g,100g,de:gläschen,Allos,allos,"[en:plant-based-foods-and-beverages, en:plant-...","en:organic,en:vegetarian,en:eu-organic,en:no-g...",Deutschland,en:germany,[Europäische Union],en:european-union
65,0756058853791,en,Bio Avocado,500g,NaN,NaN,ABC Fresh GmbH,abc-fresh-gmbh,"[en:plant-based-foods-and-beverages, en:plant-...","en:organic,en:eu-organic,en:es-eco-011-an",Germany,en:germany,[Spain],en:spain
102,4311532216507,en,NaN,NaN,NaN,NaN,Edeka,edeka,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,Germany,en:germany,[Chile],en:chile
117,00047708,en,Avocado,1,NaN,en:no-packaging,avocado,avocado,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,France,en:france,[TO 317],en:to-317
146,0191414010683,en,Avocado oil,1 lt (34 fl oz),100g,en:glass,mythical flavors TRUVA,mythical-flavors-truva,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,Denmark,en:denmark,[Turkey],en:turkey
256,0853807005828,en,Pure avocado oil,NaN,14 g (1 Tbsp),NaN,Chosen Foods,chosen-foods,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,United States,en:united-states,[Mexico],en:mexico
283,17572897,fr,NaN,3,NaN,en:net,Nosibe,nosibe,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,France,en:france,[Mexique],en:mexico
288,3700353611843,fr,NaN,500g,NaN,en:net,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,France,en:france,[Pérou],en:peru
361,00985833,en,Avacado,650 g,NaN,NaN,Marks & Spencer,marks-spencer,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,United Kingdom,en:united-kingdom,[Peru],en:peru
410,0816536011492,en,NaN,8 Oz.,1 cucharada (15ml),es:envase-de-vidrio,Kevala,kevala,"[en:plant-based-foods-and-beverages, en:plant-...","en:no-gluten,en:no-gmos,en:no-cholesterol,en:h...",Panama,en:panama,[Mexico],en:mexico


In [56]:
relevant_categories = []

for i, x in avocado_categories.iterrows():
    print(f"the  {i} row is {x} ")

the  0 row is categories_tags    en:avocadoes
Name: 0, dtype: object 
the  1 row is categories_tags    en:avocados
Name: 1, dtype: object 
the  2 row is categories_tags    en:fresh-foods
Name: 2, dtype: object 
the  3 row is categories_tags    en:fresh-vegetables
Name: 3, dtype: object 
the  4 row is categories_tags    en:fruchte
Name: 4, dtype: object 
the  5 row is categories_tags    en:fruits
Name: 5, dtype: object 
the  6 row is categories_tags    en:raw-green-avocados
Name: 6, dtype: object 
the  7 row is categories_tags    en:tropical-fruits
Name: 7, dtype: object 
the  8 row is categories_tags    en:tropische-fruchte
Name: 8, dtype: object 
the  9 row is categories_tags    en:vegetables-based-foods
Name: 9, dtype: object 
the  10 row is categories_tags    fr:hass-avocados
Name: 10, dtype: object 
